# 1. Text Generation with GPT-2
Fine-tune GPT-2 for language modeling on [WikiText-2](https://paperswithcode.com/dataset/wikitext-2).

In [2]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')
print(datasets)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [3]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [4]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(datasets["train"])

,text
0,"The 3D version ranked at No. 11 in the Japanese box office chart with a total of $ 285 @,@ 268 from a total of nine theaters . The 3D version won the Movie award for The Japanese Committee of the International 3D Society . \n"
1,"On Wheeler 's death , H.D. Sankalia of Deccan College , Pune , described him as "" well known among Old World archaeologists in the United States "" , particularly for his book Archaeology from the Earth and his studies of the Indus Valley Civilisation . In its 2013 obituary of the English archaeologist Mick Aston , British Archaeology magazine – the publication of the Council for British Archaeology – described Aston as "" the Mortimer Wheeler of our times "" because despite the strong differences between their personalities , both had done much to bring archaeology to the British public . However , writing in 2011 , Moshenska and Schadla @-@ Hall asserted that Wheeler 's reputation has not undergone significant revision among archaeologists , but that instead he had come to be remembered as "" a cartoonish and slightly eccentric figure "" whom they termed "" Naughty Morty "" . Carr described the Institute of Archaeology as "" one of the [ Wheeler ] couple 's most permanent memorials . "" \n"
2,"In January 1844 , North Carolina Representative James Iver McKay , the chairman of the Committee on Ways and Means , solicited the views of Director Patterson on the gold dollar . Patterson had more of Gobrecht 's pattern dollar struck to show to committee members , again advising against a coin that if issued would be only about a half inch ( 13 mm ) in diameter . He told Treasury Secretary John C. Spencer that the only gold coins of that size in commerce , the Spanish and Colombian half @-@ escudos , were unpopular and had not been struck for more than twenty years . This seemed to satisfy the committee as nothing more was done for the time , and when a gold dollar was proposed again in 1846 , McKay 's committee recommended against it . \n"
3,"Pernick and Wilder present examples which show that the "" clean tech revolution "" is already under way . Very large corporations such as GE , Toyota and Sharp , and investment firms such as Goldman Sachs are making multibillion @-@ dollar investments in clean technology . Emerging clean tech cities are seen to include Copenhagen , where wind power generates 20 percent of Denmark 's electricity , and Chicago , a leader in "" green "" buildings saving energy , heating and cooling costs . Statistics from the U.S. and from abroad , especially from China , India , Brazil , and Europe are presented . \n"
4,"O 'Malley considered a run for Governor of Maryland in the 2002 election , but decided not to run . He was reelected as Mayor of Baltimore in 2003 , and announced his candidacy for Governor in the 2006 election . The Baltimore Sun endorsed O 'Malley , saying : "" When he was first elected mayor in 1999 , the former two @-@ term city councilman inherited a city of rising crime , failing schools , and shrinking economic prospects . He was able to reverse course in all of these areas . "" The Washington Post criticized O 'Malley for "" not solv [ ing ] the problems of rampant crime and rough schools in Baltimore "" , but further said that "" he put a dent in them . "" . O 'Malley defeated incumbent Governor Bob Ehrlich 53 % -46 % in the November 7 , 2006 , general election . O 'Malley defeated Ehrlich in the 2010 election 56 % -42 % , receiving just over one million votes . \n"
5,"The league season concluded with an away match against divisional runners @-@ up Scunthorpe ; having gone two goals down York fought back to draw 2 – 2 with goals scored by Brobbel and Andrew . This result meant York finished the season in seventh @-@ place in League Two , and would thus play fourth @-@ placed Fleetwood in the play @-@ off semi @-@ final on the back of a 17 @-@ match unbeaten run . York lost 1 – 0 to Fleetwood in the first leg at Bootham Crescent ; the goal came from fo

In [33]:
from transformers import AutoTokenizer

model_checkpoint = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

# Set the `<EOS>` token as the padding token
# tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=128)
    return tokenizer(examples["text"], truncation=True, max_length=128)

# Remove columns to avoid the mismatched length problem
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
print(tokenized_datasets["train"][1])
print(tokenizer.decode(tokenized_datasets["train"][1]["input_ids"]))

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

{'input_ids': [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
 = Valkyria Chronicles III = 



In [34]:
# Not applied
# block_size = tokenizer.model_max_length (1024)
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of `max_length`
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [35]:
# Not applied
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4
)
print(tokenizer.decode(lm_datasets["train"][1]["input_ids"]))

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

 game and follows the " Nameless ", The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Ozawa. A large team of writers handled the script. The game's opening theme was sung by May 'n. 
 It met with positive sales in Japan, and was praised by


In [36]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [37]:
print(model.config)

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.44.2",
  "use_cache": true,
  "vocab_size": 50257
}



In [38]:
from transformers import Trainer, TrainingArguments

model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    eval_strategy = "epoch",
    num_train_epochs=3,
    learning_rate=2e-5,
    warmup_steps=500,
    weight_decay=0.01
)
print(training_args)

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
eval_use_gather_object=False,
evaluation_s

In [39]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"]
)
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.570900,3.474456
2,3.429700,3.454924
3,3.365500,3.450727


TrainOutput(global_step=5106, training_loss=3.50104506268858, metrics={'train_runtime': 588.0369, 'train_samples_per_second': 69.44, 'train_steps_per_second': 8.683, 'total_flos': 2667334385664000.0, 'train_loss': 3.50104506268858, 'epoch': 3.0})

In [40]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 31.52


In [41]:
trainer.save_model('models/wikitext2_gpt2')

In [42]:
tokenizer.save_pretrained('models/wikitext2_gpt2')

('models/wikitext2_gpt2/tokenizer_config.json',
 'models/wikitext2_gpt2/special_tokens_map.json',
 'models/wikitext2_gpt2/vocab.json',
 'models/wikitext2_gpt2/merges.txt',
 'models/wikitext2_gpt2/added_tokens.json',
 'models/wikitext2_gpt2/tokenizer.json')

In [50]:
from transformers import pipeline

# Load the model fine-tuned model
finetuned_model = pipeline("text-generation", model='models/wikitext2_gpt2', device=0)

# Set `pad_token_id` for the model or it will result in the following error 
# Setting `pad_token_id` to `eos_token_id`:{eos_token_id} for open-end generation.
finetuned_model('Can you tell me a story', pad_token_id=tokenizer.eos_token_id)

[{'generated_text': 'Can you tell me a story with a theme you can relate to me and that I\'m interested to share with you. " \n The story with the cat in this episode was inspired by a song about a baby cat that has " its soul'}]